# Lab 4: Tourism Time Series Forecasting
Machine Learning in Tourism

## Learning objectives:
1. Generate and visualize tourism time series data with realistic seasonality, trends, and external events
2. Apply multiple forecasting techniques ranging from simple naive methods to advanced models
3. Evaluate forecast accuracy using appropriate statistical metrics
4. Create ensemble forecasts by combining predictions from multiple models
5. Perform scenario analysis to assess potential future market conditions
6. Identify seasonal patterns and their impact on tourism demand
7. Develop prediction intervals to communicate forecast uncertainty
8. Extract actionable business insights from forecasting results
9. Formulate data-driven recommendations for tourism management
10. Design a monitoring framework to track forecast performance over time
11. Communicate forecasting results using effective visualizations and metrics
12. Apply forecasting to specific business functions such as capacity planning, revenue management, and marketing

## PART 1: SETUP AND DATA GENERATION

In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

# For time series analysis and forecasting
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.holtwinters import ExponentialSmoothing, SimpleExpSmoothing, Holt
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from pmdarima import auto_arima
from statsforecast import StatsForecast
from statsforecast.models import (
    AutoARIMA, SeasonalExponentialSmoothing, 
    MSTL, AutoTheta, AutoETS, TBATS
)
from sktime.forecasting.arima import AutoARIMA as SkAutoARIMA
from sktime.forecasting.naive import NaiveForecaster
from sktime.forecasting.compose import EnsembleForecaster
from sktime.forecasting.exp_smoothing import ExponentialSmoothing as SkExponentialSmoothing

# For model evaluation
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import warnings

# Set plotting style and suppress warnings
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
warnings.filterwarnings('ignore')

In [ ]:
# Generate synthetic tourism arrival data with trends, seasonality, and external effects
def generate_tourism_data(start_date='2018-01-01', periods=60, base_arrivals=10000, 
                          trend_factor=100, seasonal_amplitude=0.3, 
                          special_events=None, economic_factors=None):
    """
    Generate synthetic tourism arrival data with:
    - Upward trend
    - Monthly seasonality
    - Special events (like festivals, holidays)
    - Economic impacts (like exchange rate changes)
    
    Parameters:
    -----------
    start_date : str, starting date of the time series
    periods : int, number of months to generate
    base_arrivals : int, base level of monthly arrivals
    trend_factor : float, monthly increase in the baseline
    seasonal_amplitude : float, amplitude of seasonal fluctuations (0-1)
    special_events : dict, {date: impact_factor} for one-time events
    economic_factors : dict, {date_range: impact_factor} for economic impacts
    
    Returns:
    --------
    pandas DataFrame with dates and tourist arrivals
    """
    # Create date range
    date_range = pd.date_range(start=start_date, periods=periods, freq='MS')
    
    # Initialize arrivals with trend
    trend = np.array([base_arrivals + i * trend_factor for i in range(periods)])
    
    # Add seasonality (peaks in summer and winter holidays)
    months = np.array([d.month for d in date_range])
    seasonality = seasonal_amplitude * base_arrivals * (
        0.8 * np.sin(2 * np.pi * (months - 6) / 12) +  # Summer peak (July)
        0.5 * np.sin(2 * np.pi * (months - 12) / 12)   # Winter peak (December)
    )
    
    # Combine trend and seasonality
    arrivals = trend + seasonality
    
    # Add random noise (realistic fluctuations)
    noise = np.random.normal(0, base_arrivals * 0.05, periods)
    arrivals += noise
    
    # Create DataFrame
    tourism_df = pd.DataFrame({
        'date': date_range,
        'arrivals': arrivals.astype(int)
    })
    
    # Add special events (e.g., festivals, major conferences)
    if special_events is not None:
        for event_date, impact in special_events.items():
            idx = tourism_df[tourism_df['date'] == event_date].index
            if len(idx) > 0:
                tourism_df.loc[idx, 'arrivals'] = tourism_df.loc[idx, 'arrivals'] * impact

    # Add economic factors (e.g., exchange rate changes)
    if economic_factors is not None:
        for (start, end), impact in economic_factors.items():
            mask = (tourism_df['date'] >= start) & (tourism_df['date'] <= end)
            tourism_df.loc[mask, 'arrivals'] = tourism_df.loc[mask, 'arrivals'] * impact
    
    # Set date as index
    tourism_df.set_index('date', inplace=True)
    
    return tourism_df

# Define special events and economic factors
special_events = {
    '2019-07-01': 1.3,  # Major summer festival
    '2019-12-01': 1.2,  # Winter holiday marketing campaign
    '2020-02-01': 0.7,  # Beginning of COVID-19 concerns
    '2020-08-01': 1.15, # Local tourism campaign
    '2021-06-01': 1.25  # Post-restriction tourist promotion
}

economic_factors = {
    ('2020-03-01', '2020-08-01'): 0.4,  # COVID-19 lockdowns
    ('2020-09-01', '2020-12-01'): 0.6,  # Partial recovery
    ('2021-01-01', '2021-04-01'): 0.65, # Winter COVID wave
    ('2021-05-01', '2022-12-01'): 0.85  # Gradual recovery
}

# Generate the data
tourism_data = generate_tourism_data(
    start_date='2018-01-01', 
    periods=60,  # 5 years of monthly data
    base_arrivals=50000, 
    trend_factor=300,
    seasonal_amplitude=0.4,
    special_events=special_events,
    economic_factors=economic_factors
)

# Add year and month columns for easier analysis
tourism_data['year'] = tourism_data.index.year
tourism_data['month'] = tourism_data.index.month
tourism_data['month_name'] = tourism_data.index.month_name()

# Print the first few rows of the generated data
print("Generated Tourism Arrival Data:")
display(tourism_data.head())

In [ ]:
economic_factors

In [ ]:
tourism_data.info()

## PART 2: EXPLORATORY DATA ANALYSIS

In [ ]:
# Plot the time series
plt.figure(figsize=(14, 7))
plt.plot(tourism_data.index, tourism_data['arrivals'], marker='o', linestyle='-')
plt.title('Monthly Tourist Arrivals (2018-2022)', fontsize=15)
plt.xlabel('Date', fontsize=12)
plt.ylabel('Number of Arrivals', fontsize=12)
plt.grid(True, alpha=0.3)

# Add vertical lines and annotations for major events
for event_date, impact in special_events.items():
    event = pd.to_datetime(event_date)
    event_value = tourism_data.loc[event, 'arrivals']
    plt.axvline(x=event, color='r', linestyle='--', alpha=0.5)
    if impact > 1:
        event_type = "Positive Event"
    else:
        event_type = "Negative Event"
    plt.annotate(event_type, xy=(event, event_value), xytext=(10, 0), 
                 textcoords="offset points", fontsize=9, color='red')

# Highlight economic factor periods
for (start, end), impact in economic_factors.items():
    start_date = pd.to_datetime(start)
    end_date = pd.to_datetime(end)
    plt.axvspan(start_date, end_date, alpha=0.2, color='gray')
    middle_date = start_date + (end_date - start_date)/2
    middle_date = middle_date.to_period('M').to_timestamp() #round to month
    
    # Only annotate major periods
    if impact < 0.5:
        plt.annotate(f"Economic Impact\n({impact:.1f}x)", 
                     xy=(middle_date, tourism_data.loc[middle_date.normalize(), 'arrivals']*0.85),
                     ha='center', fontsize=10, bbox=dict(boxstyle="round,pad=0.3", 
                                                        fc="white", ec="gray", alpha=0.8))

plt.tight_layout()
plt.show()

# Analyze monthly patterns using a box plot
plt.figure(figsize=(14, 6))
sns.boxplot(data=tourism_data, x='month_name', y='arrivals', 
            order=['January', 'February', 'March', 'April', 'May', 'June', 
                   'July', 'August', 'September', 'October', 'November', 'December'])
plt.title('Monthly Distribution of Tourist Arrivals', fontsize=15)
plt.xlabel('Month', fontsize=12)
plt.ylabel('Number of Arrivals', fontsize=12)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Analyze yearly patterns
plt.figure(figsize=(10, 6))
yearly_data = tourism_data.groupby('year')['arrivals'].sum()
plt.bar(yearly_data.index, yearly_data.values)
plt.title('Yearly Tourist Arrivals', fontsize=15)
plt.xlabel('Year', fontsize=12)
plt.ylabel('Total Annual Arrivals', fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Time series decomposition
decomposition = seasonal_decompose(tourism_data['arrivals'], model='multiplicative', period=12)

plt.figure(figsize=(14, 10))

# Original data
plt.subplot(4, 1, 1)
plt.plot(decomposition.observed)
plt.title('Original Time Series', fontsize=12)
plt.grid(True, alpha=0.3)

# Trend component
plt.subplot(4, 1, 2)
plt.plot(decomposition.trend)
plt.title('Trend Component', fontsize=12)
plt.grid(True, alpha=0.3)

# Seasonal component
plt.subplot(4, 1, 3)
plt.plot(decomposition.seasonal)
plt.title('Seasonal Component', fontsize=12)
plt.grid(True, alpha=0.3)

# Residual component
plt.subplot(4, 1, 4)
plt.plot(decomposition.resid)
plt.title('Residual Component', fontsize=12)
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# ACF and PACF plots for time series analysis
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))
plot_acf(tourism_data['arrivals'].dropna(), ax=ax1, lags=24)
plot_pacf(tourism_data['arrivals'].dropna(), ax=ax2, lags=24)
plt.tight_layout()
plt.show()

## PART 3: DATA PREPARATION FOR FORECASTING

In [ ]:
# Splitting data into train and test sets
train_data = tourism_data.iloc[:-12].copy()  # Use all but the last 12 months for training
test_data = tourism_data.iloc[-12:].copy()   # Last 12 months for testing

print(f"Training data: {len(train_data)} months, from {train_data.index.min().strftime('%Y-%m')} to {train_data.index.max().strftime('%Y-%m')}")
print(f"Test data: {len(test_data)} months, from {test_data.index.min().strftime('%Y-%m')} to {test_data.index.max().strftime('%Y-%m')}")

# Visualize train-test split
plt.figure(figsize=(14, 7))
plt.plot(train_data.index, train_data['arrivals'], label='Training Data')
plt.plot(test_data.index, test_data['arrivals'], label='Test Data', color='red')
plt.title('Train-Test Split of Tourism Data', fontsize=15)
plt.xlabel('Date', fontsize=12)
plt.ylabel('Number of Arrivals', fontsize=12)
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## PART 4: FORECASTING METHODS

In [ ]:
# Function to evaluate forecasts
def evaluate_forecast(actual, forecast, model_name):
    """Evaluate forecast performance using multiple metrics"""
    rmse = np.sqrt(mean_squared_error(actual, forecast))
    mae = mean_absolute_error(actual, forecast)
    mape = np.mean(np.abs((actual - forecast) / actual)) * 100
    r2 = r2_score(actual, forecast)
    
    print(f"Model: {model_name}")
    print(f"RMSE: {rmse:.2f}")
    print(f"MAE: {mae:.2f}")
    print(f"MAPE: {mape:.2f}%")
    print(f"R²: {r2:.4f}")
    print("-" * 40)
    
    return {
        'model': model_name,
        'rmse': rmse,
        'mae': mae,
        'mape': mape,
        'r2': r2
    }

# Dictionary to store all forecasts and their evaluation metrics
all_forecasts = {}
evaluation_results = []

We evaluate different forecasting methods

In [ ]:
basic_forecats = {}
# 1. Naive Method: Last value forecast
def naive_forecast(train, test):
    """Simple naive forecast using the last value"""
    last_value = train['arrivals'].iloc[-1]
    naive_pred = pd.Series([last_value] * len(test), index=test.index)
    return naive_pred

naive_pred = naive_forecast(train_data, test_data)
basic_forecats['Naive'] = naive_pred
evaluation_results.append(evaluate_forecast(test_data['arrivals'], naive_pred, 'Naive Method'))

# 2. Seasonal Naive Method: Use value from same month in previous year
def seasonal_naive_forecast(train, test):
    """Seasonal naive forecast using the value from the same month in the previous year"""
    seasonal_naive_pred = []
    for date in test.index:
        # Get the same month from previous year
        prev_year_month = date - pd.DateOffset(months=12)
        if prev_year_month in train.index:
            seasonal_naive_pred.append(train.loc[prev_year_month, 'arrivals'])
        else:
            seasonal_naive_pred.append(train['arrivals'].iloc[-1])  # Fallback
    
    return pd.Series(seasonal_naive_pred, index=test.index)

seasonal_naive_pred = seasonal_naive_forecast(train_data, test_data)
basic_forecats['Seasonal Naive'] = seasonal_naive_pred
evaluation_results.append(evaluate_forecast(test_data['arrivals'], seasonal_naive_pred, 'Seasonal Naive Method'))

# 3. Simple Moving Average
def sma_forecast(train, test, window=3):
    """Simple moving average forecast"""
    # Calculate the moving average of the last window values
    last_window_values = train['arrivals'].iloc[-window:].mean()
    sma_pred = pd.Series([last_window_values] * len(test), index=test.index)
    return sma_pred

sma_pred = sma_forecast(train_data, test_data, window=3)
basic_forecats['SMA'] = sma_pred
evaluation_results.append(evaluate_forecast(test_data['arrivals'], sma_pred, 'Simple Moving Average (3-month)'))

# 4. Simple Exponential Smoothing
def ses_forecast(train, test):
    """Simple exponential smoothing forecast"""
    model = SimpleExpSmoothing(train['arrivals']).fit(optimized=True)
    ses_pred = model.forecast(len(test))
    return ses_pred

ses_pred = ses_forecast(train_data, test_data)
basic_forecats['SES'] = ses_pred
evaluation_results.append(evaluate_forecast(test_data['arrivals'], ses_pred, 'Simple Exponential Smoothing'))

# 5. Holt's Linear Method (Double Exponential Smoothing)
def holt_forecast(train, test):
    """Holt's linear trend method forecast"""
    model = Holt(train['arrivals']).fit(optimized=True)
    holt_pred = model.forecast(len(test))
    return holt_pred

holt_pred = holt_forecast(train_data, test_data)
basic_forecats['Holt'] = holt_pred
evaluation_results.append(evaluate_forecast(test_data['arrivals'], holt_pred, "Holt's Linear Method"))

# 6. Holt-Winters Method (Triple Exponential Smoothing)
def hw_forecast(train, test):
    """Holt-Winters method forecast with multiplicative seasonality"""
    model = ExponentialSmoothing(
        train['arrivals'],
        trend='add',
        seasonal='mul',
        seasonal_periods=12
    ).fit(optimized=True)
    
    hw_pred = model.forecast(len(test))
    return hw_pred

hw_pred = hw_forecast(train_data, test_data)
basic_forecats['Holt-Winters'] = hw_pred
evaluation_results.append(evaluate_forecast(test_data['arrivals'], hw_pred, "Holt-Winters Method"))

# 7. ARIMA Model
def arima_forecast(train, test):
    """ARIMA model forecast"""
    # Use auto_arima to find the best parameters
    auto_model = auto_arima(
        train['arrivals'],
        start_p=0, start_q=0,
        max_p=5, max_q=5,
        m=12,  # Monthly data
        seasonal=False,  # We'll use SARIMA for seasonality
        d=None,  # Let the model determine d
        trace=False,
        error_action='ignore',
        suppress_warnings=True,
        stepwise=True
    )
    
    print(f"Best ARIMA model: {auto_model.order}")
    
    # Forecast
    arima_pred = auto_model.predict(n_periods=len(test))
    return pd.Series(arima_pred, index=test.index)

arima_pred = arima_forecast(train_data, test_data)
basic_forecats['ARIMA'] = arima_pred
evaluation_results.append(evaluate_forecast(test_data['arrivals'], arima_pred, "ARIMA Model"))

# 8. SARIMA Model (Seasonal ARIMA)
def sarima_forecast(train, test):
    """SARIMA model forecast"""
    # Use auto_arima to find the best parameters
    auto_model = auto_arima(
        train['arrivals'],
        start_p=1, start_q=1,
        max_p=3, max_q=3,
        m=12,  # Monthly data
        start_P=0, start_Q=0,
        max_P=2, max_Q=2,
        d=None, D=None,  # Let the model determine d and D
        trace=False,
        error_action='ignore',
        suppress_warnings=True,
        stepwise=True,
        seasonal=True
    )
    
    print(f"Best SARIMA model: {auto_model.order}x{auto_model.seasonal_order}")
    
    # Forecast
    sarima_pred = auto_model.predict(n_periods=len(test))
    return pd.Series(sarima_pred, index=test.index)

sarima_pred = sarima_forecast(train_data, test_data)
basic_forecats['SARIMA'] = sarima_pred
evaluation_results.append(evaluate_forecast(test_data['arrivals'], sarima_pred, "SARIMA Model"))


In [ ]:
# Function to plot forecasts
def plot_forecasts(train, test, forecasts_dict, models_to_plot=None):
    """
    Plot actual vs. forecast values for selected models
    
    Parameters:
    -----------
    train : pandas DataFrame with training data
    test : pandas DataFrame with test data
    forecasts_dict : dict of model forecasts
    models_to_plot : list of model names to plot (if None, plot all)
    """
    if models_to_plot is None:
        models_to_plot = list(forecasts_dict.keys())
    
    plt.figure(figsize=(15, 8))
    
    # Plot training data
    plt.plot(train.index, train['arrivals'], label='Training Data', color='black', linewidth=2)
    
    # Plot test data
    plt.plot(test.index, test['arrivals'], label='Actual (Test Data)', color='blue', 
             linewidth=2, linestyle='--', marker='o')
    
    # Plot forecasts
    colors = plt.cm.tab10.colors
    for i, model_name in enumerate(models_to_plot):
        if model_name in forecasts_dict:
            plt.plot(test.index, forecasts_dict[model_name], 
                     label=f'Forecast: {model_name}', 
                     color=colors[i % len(colors)],
                     linewidth=1.5, marker='x')
    
    plt.title('Tourism Arrivals: Actual vs. Forecasts', fontsize=15)
    plt.xlabel('Date', fontsize=12)
    plt.ylabel('Number of Arrivals', fontsize=12)
    plt.legend(loc='best')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

# Plot all forecasts so far
plot_forecasts(train_data, test_data, basic_forecats)

## PART 5: ADVANCED FORECASTING TECHNIQUES

In [ ]:
advanced_forecasts = {}
# Using StatsForecast for advanced models
def advanced_forecasting(train, test):
    """Advanced forecasting methods using StatsForecast library"""
    # Prepare data in the format required by StatsForecast
    Y_df = train.reset_index()[['date', 'arrivals']].rename(columns={'date': 'ds', 'arrivals': 'y'})
    Y_df['unique_id'] = 'tourism'  # StatsForecast requires a unique_id column
    
    # Define models
    models = [
        AutoETS(season_length=12),  # Exponential Smoothing State Space Model
        TBATS(season_length=12),   # Trigonometric Seasonality, Box-Cox transform, ARMA errors, Trend, and Seasonal components
        MSTL(season_length=12)     # Multiple Seasonal-Trend decomposition using LOESS
    ]
    
    # Initialize StatsForecast
    sf = StatsForecast(
        models=models,
        freq='MS',  # Month Start frequency
        n_jobs=-1   # Use all available cores
    )
    
    # Fit models and forecast
    sf.fit(Y_df)
    h = len(test)
    forecasts = sf.predict(h=h)
    
    # Extract forecasts for each model
    ets_forecast = forecasts[['ds', 'AutoETS']].set_index('ds')['AutoETS']
    tbats_forecast = forecasts[['ds', 'TBATS']].set_index('ds')['TBATS']
    mstl_forecast = forecasts[['ds', 'MSTL']].set_index('ds')['MSTL']
    
    return ets_forecast, tbats_forecast, mstl_forecast

# Run advanced forecasting
try:
    ets_pred, tbats_pred, mstl_pred = advanced_forecasting(train_data, test_data)
    
    # Add to advanced_forecasts dictionary
    advanced_forecasts['ETS'] = ets_pred
    advanced_forecasts['TBATS'] = tbats_pred
    advanced_forecasts['MSTL'] = mstl_pred
    
    # Evaluate advanced forecasts
    evaluation_results.append(evaluate_forecast(test_data['arrivals'], ets_pred, "ETS (Exponential Smoothing State Space)"))
    evaluation_results.append(evaluate_forecast(test_data['arrivals'], tbats_pred, "TBATS"))
    evaluation_results.append(evaluate_forecast(test_data['arrivals'], mstl_pred, "MSTL"))
except Exception as e:
    print(f"Advanced forecasting encountered an error: {e}")
    print("Continuing with available models...")


In [ ]:
# Plot advanced forecasts
plot_forecasts(train_data, test_data, advanced_forecasts)

## PART 6: ENSEMBLE FORECASTING

In [ ]:
# Create an ensemble forecast (simple average of best performing methods)
def create_ensemble(forecasts_dict, methods=None):
    """
    Create an ensemble forecast by averaging selected methods
    
    Parameters:
    -----------
    forecasts_dict : dict of forecast series
    methods : list of method names to include (if None, use all)
    
    Returns:
    --------
    pandas Series with ensemble forecast
    """
    if methods is None:
        methods = list(forecasts_dict.keys())
    
    # Extract selected forecasts
    selected_forecasts = [forecasts_dict[method] for method in methods if method in forecasts_dict]
    
    # Create a DataFrame from the forecasts
    ensemble_df = pd.concat(selected_forecasts, axis=1)
    ensemble_df.columns = methods
    
    # Average the forecasts
    ensemble_forecast = ensemble_df.mean(axis=1)
    
    return ensemble_forecast

# Create ensemble forecasts
# join all forecasts into the all_forecasts dict
all_forecasts = {**basic_forecats, **advanced_forecasts}
# 1. Simple average of all methods
all_methods_ensemble = create_ensemble(all_forecasts)
# Add ensemble forecast to the dictionary
all_forecasts['Ensemble (All)'] = all_methods_ensemble
evaluation_results.append(evaluate_forecast(test_data['arrivals'], all_methods_ensemble, "Ensemble (All Methods)"))
# 2. Best methods ensemble (identify top 3 based on RMSE)
top_methods = sorted(evaluation_results, key=lambda x: x['rmse'])[:3]
top_method_names = [result['model'] for result in top_methods]
print(f"Top performing methods: {', '.join(top_method_names)}")
method_names = [e['model'] for e in evaluation_results]
top_method_indices = [method_names.index(name) for name in top_method_names if name in method_names]
top_forecast_keys = [list(all_forecasts.keys())[i] for i in top_method_indices]

best_methods_ensemble = create_ensemble(all_forecasts, top_forecast_keys)
all_forecasts['Ensemble (Best)'] = best_methods_ensemble
evaluation_results.append(evaluate_forecast(test_data['arrivals'], best_methods_ensemble, "Ensemble (Best Methods)"))

## PART 7: VISUALIZE BEST FORECAST RESULTS

In [ ]:
# Plot only the top performing models and ensemble
top_methods_to_plot = top_forecast_keys + ['Ensemble (Best)']
plot_forecasts(train_data, test_data, all_forecasts, top_methods_to_plot)

# Create a summary table of evaluation metrics
metrics_df = pd.DataFrame(evaluation_results)
metrics_df = metrics_df.sort_values('rmse')
print("\nModel Performance Summary (Sorted by RMSE):")
display(metrics_df)

# Visualize metrics comparison
plt.figure(figsize=(12, 6))
sns.barplot(x='model', y='rmse', data=metrics_df)
plt.title('RMSE by Model', fontsize=15)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

## PART 8: FUTURE FORECASTING AND SCENARIO ANALYSIS

In [ ]:
# Generate future dates for forecasting
last_date = tourism_data.index[-1]
future_dates = pd.date_range(start=last_date + pd.DateOffset(months=1), periods=24, freq='MS')
future_df = pd.DataFrame(index=future_dates)
future_df['year'] = future_df.index.year
future_df['month'] = future_df.index.month
future_df['date'] = future_df.index

# Function to generate future scenarios
def generate_scenarios(base_model, train_data, future_dates, scenarios):
    """
    Generate forecast scenarios for different conditions
    
    Parameters:
    -----------
    base_model : function that produces forecast
    train_data : pandas DataFrame with historical data
    future_dates : pandas DatetimeIndex with future dates
    scenarios : dict of scenario descriptions and adjustment factors
    
    Returns:
    --------
    dict of pandas Series with forecasts for each scenario
    """
    scenario_forecasts = {}
    
    # Generate base forecast using the best model (or ensemble)
    if base_model == 'Holt-Winters':
        model = ExponentialSmoothing(
            train_data['arrivals'],
            trend='add',
            seasonal='mul',
            seasonal_periods=12
        ).fit(optimized=True)
        base_forecast = model.forecast(len(future_dates))
    elif base_model == 'SARIMA':
        auto_model = auto_arima(
            train_data['arrivals'],
            start_p=1, start_q=1,
            max_p=3, max_q=3,
            m=12,
            start_P=0, start_Q=0,
            max_P=2, max_Q=2,
            d=None, D=None,
            trace=False,
            error_action='ignore',
            suppress_warnings=True,
            stepwise=True,
            seasonal=True
        )
        base_forecast = pd.Series(auto_model.predict(n_periods=len(future_dates)), index=future_dates)
    elif base_model == 'Ensemble':
        # Use the best individual model for base forecast
        if 'Holt-Winters' in top_method_names:
            model = ExponentialSmoothing(
                train_data['arrivals'],
                trend='add',
                seasonal='mul',
                seasonal_periods=12
            ).fit(optimized=True)
            base_forecast = model.forecast(len(future_dates))
        else:
            auto_model = auto_arima(
                train_data['arrivals'],
                seasonal=True,
                m=12,
                trace=False,
                error_action='ignore',
                suppress_warnings=True
            )
            base_forecast = pd.Series(auto_model.predict(n_periods=len(future_dates)), index=future_dates)
    else:
        # Default to Holt-Winters if specified model not recognized
        model = ExponentialSmoothing(
            train_data['arrivals'],
            trend='add',
            seasonal='mul',
            seasonal_periods=12
        ).fit(optimized=True)
        base_forecast = model.forecast(len(future_dates))
    
    # Store the base forecast
    scenario_forecasts['Base Forecast'] = base_forecast
    
    # Generate adjusted forecasts for each scenario
    for scenario_name, adjustments in scenarios.items():
        adjusted_forecast = base_forecast.copy()
        
        # Apply each adjustment
        for period, factor in adjustments.items():
            if isinstance(period, tuple) and len(period) == 2:
                # Date range adjustment
                start_date, end_date = period
                mask = (adjusted_forecast.index >= start_date) & (adjusted_forecast.index <= end_date)
                adjusted_forecast[mask] *= factor
            elif isinstance(period, str):
                # Single date or generic condition
                if period.startswith('month_'):
                    # Apply to specific months
                    month = int(period.split('_')[1])
                    mask = adjusted_forecast.index.month == month
                    adjusted_forecast[mask] *= factor
                elif period == 'summer':
                    # Apply to summer months (June-August)
                    summer_mask = adjusted_forecast.index.month.isin([6, 7, 8])
                    adjusted_forecast[summer_mask] *= factor
                elif period == 'winter':
                    # Apply to winter months (December-February)
                    winter_mask = adjusted_forecast.index.month.isin([12, 1, 2])
                    adjusted_forecast[winter_mask] *= factor
                elif period == 'all':
                    # Apply to all forecast periods
                    adjusted_forecast *= factor
            else:
                # Specific month index adjustment
                if period < len(adjusted_forecast):
                    adjusted_forecast.iloc[period] *= factor
        
        scenario_forecasts[scenario_name] = adjusted_forecast
    
    return scenario_forecasts

# Define scenarios
scenarios = {
    'Optimistic Recovery': {
        'all': 1.15,  # Overall 15% increase
        'summer': 1.25,  # Additional 25% increase in summer months
        (future_dates[0], future_dates[5]): 1.05  # Additional 5% in first 6 months
    },
    'Conservative Recovery': {
        'all': 1.05,  # Overall 5% increase
        'summer': 1.1,  # Additional 10% increase in summer
        'winter': 0.95  # 5% decrease in winter
    },
    'New Market Entry': {
        'all': 1.2,  # Overall 20% increase
        'month_6': 1.3,  # Additional 30% increase in June (new flight routes)
        'month_7': 1.3,  # Additional 30% increase in July
        (future_dates[12], future_dates[23]): 1.15  # Additional 15% in second year
    },
    'Economic Downturn': {
        'all': 0.85,  # Overall 15% decrease
        'summer': 0.9,  # Summer less affected (-10%)
        'winter': 0.8  # Winter more affected (-20%)
    },
    'Enhanced Marketing': {
        'all': 1.1,  # Overall 10% increase
        (future_dates[0], future_dates[2]): 1.2,  # Additional 20% in first 3 months (campaign start)
        'month_6': 1.25,  # Additional 25% in June
        'month_12': 1.15  # Additional 15% in December
    }
}

# Generate scenarios using the best method
best_model = metrics_df.iloc[0]['model'].split()[0]
print(f"Using {best_model} as base model for scenario forecasting")
scenario_forecasts = generate_scenarios(best_model, tourism_data, future_dates, scenarios)

# Plot scenario forecasts
plt.figure(figsize=(15, 8))

# Plot historical data
plt.plot(tourism_data.index, tourism_data['arrivals'], 
         label='Historical Data', color='black', linewidth=2)

# Plot scenarios
colors = plt.cm.Dark2.colors
for i, (scenario_name, forecast) in enumerate(scenario_forecasts.items()):
    plt.plot(forecast.index, forecast, 
             label=f'Scenario: {scenario_name}', 
             color=colors[i % len(colors)],
             linewidth=2)

plt.title('Tourism Arrivals: Future Scenarios (24-Month Forecast)', fontsize=15)
plt.xlabel('Date', fontsize=12)
plt.ylabel('Number of Arrivals', fontsize=12)
plt.legend(loc='best')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Create a summary table of scenario forecasts
scenario_summary = pd.DataFrame({
    'Scenario': list(scenario_forecasts.keys()),
    'Avg Monthly Arrivals': [forecast.mean() for forecast in scenario_forecasts.values()],
    'Peak Month Arrivals': [forecast.max() for forecast in scenario_forecasts.values()],
    'Total 2-Year Arrivals': [forecast.sum() for forecast in scenario_forecasts.values()],
    'Year 1 Growth': [(forecast.iloc[:12].mean() / tourism_data.iloc[-12:]['arrivals'].mean() - 1) * 100 
                      for forecast in scenario_forecasts.values()],
    'Year 2 Growth': [(forecast.iloc[12:].mean() / forecast.iloc[:12].mean() - 1) * 100 
                      for forecast in scenario_forecasts.values()]
})

# Format the numeric columns
scenario_summary['Avg Monthly Arrivals'] = scenario_summary['Avg Monthly Arrivals'].round(0).astype(int)
scenario_summary['Peak Month Arrivals'] = scenario_summary['Peak Month Arrivals'].round(0).astype(int)
scenario_summary['Total 2-Year Arrivals'] = scenario_summary['Total 2-Year Arrivals'].round(0).astype(int)
scenario_summary['Year 1 Growth'] = scenario_summary['Year 1 Growth'].round(1)
scenario_summary['Year 2 Growth'] = scenario_summary['Year 2 Growth'].round(1)

print("\nScenario Forecast Summary:")
display(scenario_summary)

## PART 9: FINAL MODEL SELECTION AND BUSINESS RECOMMENDATIONS

In [ ]:
# Select the best model based on performance metrics
best_model_name = metrics_df.iloc[0]['model']
print(f"\nBest performing model: {best_model_name}")

# Create a final forecast for the next 24 months using the best model
if "Holt-Winters" in best_model_name:
    final_model = ExponentialSmoothing(
        tourism_data['arrivals'],
        trend='add',
        seasonal='mul',
        seasonal_periods=12
    ).fit(optimized=True)
    final_forecast = final_model.forecast(24)
elif "SARIMA" in best_model_name:
    final_model = auto_arima(
        tourism_data['arrivals'],
        seasonal=True,
        m=12,
        trace=False,
        error_action='ignore',
        suppress_warnings=True
    )
    final_forecast = pd.Series(final_model.predict(n_periods=24), index=future_dates)
elif "Ensemble" in best_model_name:
    # Use the ensemble approach for final forecast
    final_forecast = pd.Series(0, index=future_dates)
    for method in top_method_names:
        if method == 'Holt-Winters':
            model = ExponentialSmoothing(
                tourism_data['arrivals'],
                trend='add',
                seasonal='mul',
                seasonal_periods=12
            ).fit(optimized=True)
            method_forecast = model.forecast(24)
        elif method == 'SARIMA':
            model = auto_arima(
                tourism_data['arrivals'],
                seasonal=True,
                m=12,
                trace=False,
                error_action='ignore',
                suppress_warnings=True
            )
            method_forecast = pd.Series(model.predict(n_periods=24), index=future_dates)
        elif method == 'ETS':
            # Simplified ETS forecast
            model = ExponentialSmoothing(
                tourism_data['arrivals'],
                trend='add',
                damped_trend=True,
                seasonal='mul',
                seasonal_periods=12
            ).fit(optimized=True)
            method_forecast = model.forecast(24)
        else:
            # Default to Holt-Winters if method not recognized
            model = ExponentialSmoothing(
                tourism_data['arrivals'],
                trend='add',
                seasonal='mul',
                seasonal_periods=12
            ).fit(optimized=True)
            method_forecast = model.forecast(24)
        
        final_forecast += method_forecast
    
    final_forecast = final_forecast / len(top_method_names)
else:
    # Default to Holt-Winters if best model not recognized
    final_model = ExponentialSmoothing(
        tourism_data['arrivals'],
        trend='add',
        seasonal='mul',
        seasonal_periods=12
    ).fit(optimized=True)
    final_forecast = final_model.forecast(24)

# Calculate prediction intervals (simplified approach)
def calculate_prediction_intervals(forecast, history, confidence=0.95):
    """Calculate prediction intervals based on historical forecast errors"""
    # For simplicity, we'll use the RMSE from the test period to estimate error range
    best_rmse = metrics_df.iloc[0]['rmse']
    
    # Z-score for the confidence interval (e.g., 1.96 for 95%)
    z = {0.90: 1.645, 0.95: 1.96, 0.99: 2.576}.get(confidence, 1.96)
    
    # Calculate intervals
    lower_bound = forecast - z * best_rmse
    upper_bound = forecast + z * best_rmse
    
    return lower_bound, upper_bound

# Calculate prediction intervals
lower_bound, upper_bound = calculate_prediction_intervals(final_forecast, tourism_data['arrivals'])

# Visualize final forecast with prediction intervals
plt.figure(figsize=(15, 8))

# Plot historical data
plt.plot(tourism_data.index, tourism_data['arrivals'], 
         label='Historical Data', color='black', linewidth=2)

# Plot forecast and intervals
plt.plot(final_forecast.index, final_forecast, 
         label='Final Forecast', color='blue', linewidth=2.5)
plt.fill_between(final_forecast.index, lower_bound, upper_bound, 
                color='blue', alpha=0.2, label='95% Prediction Interval')

# Improve visual appeal
plt.title('Tourism Arrivals: 24-Month Forecast with Prediction Intervals', fontsize=15)
plt.xlabel('Date', fontsize=12)
plt.ylabel('Number of Arrivals', fontsize=12)
plt.legend(loc='best')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## PART 10: ACTIONABLE BUSINESS RECOMMENDATIONS

In [ ]:
# Create a DataFrame for monthly forecasts with key metrics
forecast_df = pd.DataFrame({
    'Date': final_forecast.index,
    'Forecast': final_forecast.values,
    'Lower Bound': lower_bound.values,
    'Upper Bound': upper_bound.values
})

forecast_df['Month'] = forecast_df['Date'].dt.month
forecast_df['Month Name'] = forecast_df['Date'].dt.month_name()
forecast_df['Year'] = forecast_df['Date'].dt.year
forecast_df['YearMonth'] = forecast_df['Date'].dt.strftime('%Y-%m')

# Calculate month-over-month and year-over-year growth
forecast_df['MoM Growth %'] = forecast_df['Forecast'].pct_change() * 100

# Calculate YoY growth (only for the second year)
for i in range(12, len(forecast_df)):
    forecast_df.loc[i, 'YoY Growth %'] = (
        (forecast_df.loc[i, 'Forecast'] / forecast_df.loc[i-12, 'Forecast'] - 1) * 100
    )

# Identify peak months
forecast_df['Peak Flag'] = False
for year in forecast_df['Year'].unique():
    year_data = forecast_df[forecast_df['Year'] == year]
    max_month = year_data['Forecast'].idxmax()
    forecast_df.loc[max_month, 'Peak Flag'] = True

# Identify trough months
forecast_df['Trough Flag'] = False
for year in forecast_df['Year'].unique():
    year_data = forecast_df[forecast_df['Year'] == year]
    min_month = year_data['Forecast'].idxmin()
    forecast_df.loc[min_month, 'Trough Flag'] = True

# Calculate seasonality factors
avg_monthly_forecast = forecast_df.groupby('Month')['Forecast'].mean()
overall_avg = avg_monthly_forecast.mean()
seasonality_factors = avg_monthly_forecast / overall_avg

# Add seasonality factors to the forecast DataFrame
forecast_df['Seasonality Factor'] = forecast_df['Month'].map(
    {month: factor for month, factor in seasonality_factors.items()}
)

# Mark high season and low season
forecast_df['Season'] = 'Regular'
high_season_months = seasonality_factors[seasonality_factors >= 1.1].index.tolist()
low_season_months = seasonality_factors[seasonality_factors <= 0.9].index.tolist()
forecast_df.loc[forecast_df['Month'].isin(high_season_months), 'Season'] = 'High'
forecast_df.loc[forecast_df['Month'].isin(low_season_months), 'Season'] = 'Low'

# Print the detailed forecast table
print("\nDetailed Monthly Forecast:")
display(forecast_df[['YearMonth', 'Forecast', 'Lower Bound', 'Upper Bound', 
                  'MoM Growth %', 'YoY Growth %', 'Season', 'Peak Flag', 'Trough Flag']]
      .sort_values('YearMonth')
      .head(24))

# Visualize seasonality patterns
plt.figure(figsize=(12, 6))
seasonality_df = pd.DataFrame({
    'Month': seasonality_factors.index,
    'Factor': seasonality_factors.values
})
seasonality_df['Month Name'] = seasonality_df['Month'].apply(
    lambda x: {1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Apr', 5: 'May', 6: 'Jun', 
               7: 'Jul', 8: 'Aug', 9: 'Sep', 10: 'Oct', 11: 'Nov', 12: 'Dec'}[x]
)
sns.barplot(x='Month Name', y='Factor', data=seasonality_df, order=[
    'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
    'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'
])
plt.axhline(y=1, color='red', linestyle='--')
plt.title('Tourism Seasonality Factors', fontsize=15)
plt.xlabel('Month', fontsize=12)
plt.ylabel('Seasonality Factor', fontsize=12)
plt.tight_layout()
plt.show()

# Calculate key business metrics
total_annual_visitors = {
    year: forecast_df[forecast_df['Year'] == year]['Forecast'].sum()
    for year in forecast_df['Year'].unique()
}

average_monthly_visitors = {
    year: forecast_df[forecast_df['Year'] == year]['Forecast'].mean()
    for year in forecast_df['Year'].unique()
}

peak_month_data = forecast_df[forecast_df['Peak Flag']].copy()
peak_month_data['Year'] = peak_month_data['Date'].dt.year
peak_months = {
    year: row['Month Name']
    for year, row in peak_month_data.groupby('Year').first().iterrows()
}

trough_month_data = forecast_df[forecast_df['Trough Flag']].copy()
trough_month_data['Year'] = trough_month_data['Date'].dt.year
trough_months = {
    year: row['Month Name']
    for year, row in trough_month_data.groupby('Year').first().iterrows()
}

high_season_months_names = [
    {1: 'January', 2: 'February', 3: 'March', 4: 'April', 5: 'May', 6: 'June', 
     7: 'July', 8: 'August', 9: 'September', 10: 'October', 11: 'November', 12: 'December'}[month]
    for month in high_season_months
]

low_season_months_names = [
    {1: 'January', 2: 'February', 3: 'March', 4: 'April', 5: 'May', 6: 'June', 
     7: 'July', 8: 'August', 9: 'September', 10: 'October', 11: 'November', 12: 'December'}[month]
    for month in low_season_months
]

peak_to_trough_ratio = forecast_df[forecast_df['Peak Flag']]['Forecast'].mean() / \
                       forecast_df[forecast_df['Trough Flag']]['Forecast'].mean()

year_over_year_growth = ((total_annual_visitors[list(total_annual_visitors.keys())[1]] / 
                         total_annual_visitors[list(total_annual_visitors.keys())[0]]) - 1) * 100

# Print business metrics summary
print("\n==== TOURISM FORECAST BUSINESS METRICS ====")
print("\nAnnual Visitor Forecasts:")
for year, visitors in total_annual_visitors.items():
    print(f"  {year}: {visitors:,.0f} visitors")

print("\nAverage Monthly Visitors:")
for year, visitors in average_monthly_visitors.items():
    print(f"  {year}: {visitors:,.0f} visitors per month")

print("\nPeak Months:")
for year, month in peak_months.items():
    print(f"  {year}: {month}")

print("\nTrough Months:")
for year, month in trough_months.items():
    print(f"  {year}: {month}")

print(f"\nHigh Season Months: {', '.join(high_season_months_names)}")
print(f"Low Season Months: {', '.join(low_season_months_names)}")
print(f"Peak-to-Trough Ratio: {peak_to_trough_ratio:.2f}x")
print(f"Year-over-Year Growth: {year_over_year_growth:.1f}%")

## PART 11: BUSINESS RECOMMENDATIONS BASED ON FORECAST

In [ ]:
print("\n==== ACTIONABLE BUSINESS RECOMMENDATIONS ====\n")

# Capacity Planning Recommendations
print("1. CAPACITY PLANNING RECOMMENDATIONS:")
print(f"   ✓ Prepare for peak visitor volumes during {', '.join(high_season_months_names)}")
print(f"   ✓ Maximum expected arrivals: {forecast_df['Forecast'].max():,.0f} visitors (in {forecast_df.loc[forecast_df['Forecast'].idxmax(), 'Month Name']})")
print(f"   ✓ Average high season capacity required: {forecast_df[forecast_df['Season'] == 'High']['Forecast'].mean():,.0f} visitors per month")
print(f"   ✓ Consider temporary staffing increases of {(forecast_df[forecast_df['Season'] == 'High']['Forecast'].mean() / forecast_df[forecast_df['Season'] == 'Regular']['Forecast'].mean() - 1) * 100:.0f}% during high season months")

# Revenue Management Recommendations
print("\n2. REVENUE MANAGEMENT STRATEGIES:")
high_season_premium = 15  # Example premium percentage for high season
low_season_discount = 20  # Example discount percentage for low season
print(f"   ✓ Implement dynamic pricing with {high_season_premium}% premium during high season ({', '.join(high_season_months_names)})")
print(f"   ✓ Offer {low_season_discount}% discounts during low season ({', '.join(low_season_months_names)}) to stimulate demand")
print(f"   ✓ Focus on high-margin products/services during peak months ({', '.join([month for year, month in peak_months.items()])})")
print(f"   ✓ Create special packages to increase average spend during shoulder seasons")

# Marketing Recommendations
marketing_lead_time = 3  # months ahead of peak season to begin marketing
print("\n3. MARKETING CAMPAIGN TIMING:")
print(f"   ✓ Launch high season campaigns {marketing_lead_time} months before peak periods")
for year, month in peak_months.items():
    peak_month_idx = list({1: 'January', 2: 'February', 3: 'March', 4: 'April', 5: 'May', 6: 'June', 
                         7: 'July', 8: 'August', 9: 'September', 10: 'October', 11: 'November', 12: 'December'}.keys())[list({1: 'January', 2: 'February', 3: 'March', 4: 'April', 5: 'May', 6: 'June', 
                         7: 'July', 8: 'August', 9: 'September', 10: 'October', 11: 'November', 12: 'December'}.values()).index(month)]
    campaign_month_idx = (peak_month_idx - marketing_lead_time) % 12
    if campaign_month_idx == 0:
        campaign_month_idx = 12
    campaign_month = {1: 'January', 2: 'February', 3: 'March', 4: 'April', 5: 'May', 6: 'June', 
                      7: 'July', 8: 'August', 9: 'September', 10: 'October', 11: 'November', 12: 'December'}[campaign_month_idx]
    print(f"   ✓ For {year} peak ({month}): Begin campaign in {campaign_month}")
print(f"   ✓ Allocate approximately {(forecast_df[forecast_df['Season'] == 'High']['Forecast'].sum() / forecast_df['Forecast'].sum()) * 100:.0f}% of annual marketing budget to high season periods")
print(f"   ✓ Create targeted campaigns for low season months to highlight unique experiences and value")

# Product Development Recommendations
print("\n4. PRODUCT DEVELOPMENT:")
print(f"   ✓ Develop weather-independent attractions to reduce {peak_to_trough_ratio:.1f}x seasonal volatility")
print(f"   ✓ Create special events during {', '.join(low_season_months_names)} to boost low season visitation")
print(f"   ✓ Develop premium experiences for peak months to maximize revenue potential")
print(f"   ✓ Consider business tourism and conference packages for traditionally leisure-low periods")

# Strategic Partnership Recommendations
print("\n5. STRATEGIC PARTNERSHIPS:")
print(f"   ✓ Partner with airlines for increased capacity during {', '.join(high_season_months_names)}")
print(f"   ✓ Develop joint campaigns with complementary tourism services (accommodations, attractions)")
print(f"   ✓ Create package deals with transportation providers during low season")
print(f"   ✓ Explore cross-selling opportunities with nearby destinations for extended stays")

# Risk Management Recommendations
print("\n6. RISK MANAGEMENT:")
risk_threshold = forecast_df['Upper Bound'].mean() * 1.1  # 10% above upper bound average
print(f"   ✓ Prepare contingency plans for visitor numbers exceeding {risk_threshold:,.0f}")
print(f"   ✓ Develop strategies to mitigate {(forecast_df['Upper Bound'] - forecast_df['Lower Bound']).mean() / forecast_df['Forecast'].mean() * 100:.0f}% forecast uncertainty")
print(f"   ✓ Monitor early indicators (web searches, booking patterns) to adjust forecasts quarterly")
print(f"   ✓ Create flexible capacity plans that can scale with ±20% forecast deviation")

# Long-term Planning Recommendations
print("\n7. LONG-TERM PLANNING:")
trend_slope = np.polyfit(range(len(forecast_df)), forecast_df['Forecast'], 1)[0]
annual_trend_percent = (trend_slope * 12 / forecast_df['Forecast'].iloc[0]) * 100
print(f"   ✓ Plan for {annual_trend_percent:.1f}% annual growth trend in tourism arrivals")
print(f"   ✓ Invest in infrastructure to accommodate growing peak season demand")
print(f"   ✓ Develop sustainability initiatives to manage growth in environmentally sensitive areas")
print(f"   ✓ Consider capacity constraints and carrying capacity in popular attractions and sites")

## PART 12: MONITORING AND UPDATES
==== FORECAST MONITORING PLAN ====

1. KPIs TO MONITOR:
   ✓ Monthly visitor arrivals vs. forecast (±10% tolerance)  
   ✓ Booking pace (30/60/90 days out)  
   ✓ Year-over-year growth rates  
   ✓ Seasonality patterns vs. historical norms  

2. UPDATE FREQUENCY:
   ✓ Refresh short-term forecasts monthly  
   ✓ Update medium-term forecasts quarterly  
   ✓ Recalibrate long-term forecasts annually  

3. EARLY WARNING INDICATORS:
   ✓ Search trends for destination (leading indicator)  
   ✓ Advance booking patterns  
   ✓ Competitor pricing changes  
   ✓ Transportation capacity changes  

4. ADAPTATION TRIGGERS:
   ✓ >15% deviation from forecast for 2 consecutive months  
   ✓ Significant economic events affecting source markets  
   ✓ Major changes in travel regulations or restrictions  
   ✓ Competitive landscape shifts  

## END OF TOURISM FORECASTING ANALYSIS
This analysis demonstrates comprehensive time series forecasting approaches  
with business-relevant insights and actionable recommendations.  